In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
os.chdir("/home/yw699/codes/LLM-Hallu")
sys.path.append(os.path.abspath("src"))

## 读取配置文件

In [3]:
import yaml
with open("configs/experiment_config1.yaml", "r") as file:
    config = yaml.safe_load(file)

## Dataset加载与格式化

In [4]:
from utils import *
from dataset import Dataset
dataset_loader = Dataset(config)
train_dataset, validation_dataset = dataset_loader.load_data()
print(validation_dataset[2])  

if not isinstance(train_dataset, list):
        logging.info('Train dataset: %s', train_dataset)

answerable_indices, unanswerable_indices = split_dataset(train_dataset)


if config["dataset"]['answerable_only']:
        unanswerable_indices = []
        val_answerable, val_unanswerable = split_dataset(validation_dataset)
        del val_unanswerable
        validation_dataset = [validation_dataset[i] for i in val_answerable]

{'id': '56ddde6b9a695914005b962a', 'title': 'Normans', 'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.', 'question': 'From which countries did the Norse originate?', 'answers': {'text': ['Denmark, Iceland and Norway', 'Denmark, Iceland and Norway', 'Denmark, Iceland and Norway', 'Denmark, Ice

## Prompt Engineer

In [5]:
from prompt_engineer import PromptGenerator

promptgenerator = PromptGenerator(config,validation_dataset)

In [6]:
prompt = promptgenerator.generate_template_prompt_by_id(1)
print(prompt)
print(len(promptgenerator.get_unused_indices()))


USER: When were the Normans in Normandy?
ASSISTANT:
5927


In [7]:
prompts = promptgenerator.generate_template_prompts_by_count(2)
print(prompt)
print(len(promptgenerator.get_unused_indices()))

USER: When were the Normans in Normandy?
ASSISTANT:
5925


In [8]:
import random
indices = random.sample(promptgenerator.get_unused_indices(),2)
print(indices)
prompts =promptgenerator.generate_template_prompts_from_indices(indices)
len(promptgenerator.get_unused_indices())

[843, 3422]


5923

In [9]:
few_shot_prompt = promptgenerator.construct_fewshot_prompt_from_indices()
print(few_shot_prompt)
print(len(promptgenerator.get_unused_indices()))

Answer the following question as briefly as possible.
Context: Islamists have asked the question, "If Islam is a way of life, how can we say that those who want to live by its principles in legal, social, political, economic, and political spheres of life are not Muslims, but Islamists and believe in Islamism, not [just] Islam?" Similarly, a writer for the International Crisis Group maintains that "the conception of 'political Islam'" is a creation of Americans to explain the Iranian Islamic Revolution and apolitical Islam was a historical fluke of the "short-lived era of the heyday of secular Arab nationalism between 1945 and 1970", and it is quietist/non-political Islam, not Islamism, that requires explanation.
Question: What term do Islamists think should be applied to them?
Answer: Muslims

Answer the following question as briefly as possible.
Context: Jacksonville, like most large cities in the United States, suffered from negative effects of rapid urban sprawl after World War II.

## model

In [10]:
from models import HuggingfaceModel
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [11]:
huggingface_model = HuggingfaceModel(config)

KeyboardInterrupt: 

In [ ]:
#temperature=1.0,return_full=False
output_text = huggingface_model.predict(prompt)
print("Generated Text > \n", output_text)

In [ ]:
import math

user_tag = "USER:"
assistant_tag = "ASSISTANT:"
input = "What is the capital of France?"
v = huggingface_model.get_p_true(input,"Washington")
probability = math.exp(v)
print(f"Probability: {probability:.4f}")


v = huggingface_model.get_p_true(input,"Paris")
probability = math.exp(v)
print(f"Probability: {probability:.4f}")


v = huggingface_model.get_p_true(input,"Beijing")
probability = math.exp(v)
print(f"Probability: {probability:.4f}")

v = huggingface_model.get_p_true(input,"\n")
probability = math.exp(v)
print(f"Probability: {probability:.4f}")

In [ ]:
input_text = prompts[1]

output_text,log = huggingface_model.predict(input_text)
print("<Generated Text>:", output_text)
print(log)

In [ ]:
from samplers import Sampler

sampler = Sampler(prompt,huggingface_model,sample_method="simple")

In [ ]:
sampler.sample(8)
sampler.show_all_result()